In [2]:
import pandas as pd

pets_df = pd.read_json('pet_supplies_100.json', lines=True)

In [3]:
pets_df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A14CK12J7C7JRK,1223000893,Consumer in NorCal,"[0, 0]",I purchased the Trilogy with hoping my two cat...,3,Nice Distraction for my cats for about 15 minutes,1294790400,"01 12, 2011"
1,A39QHP5WLON5HV,1223000893,Melodee Placial,"[0, 0]",There are usually one or more of my cats watch...,5,Entertaining for my cats,1379116800,"09 14, 2013"
2,A2CR37UY3VR7BN,1223000893,Michelle Ashbery,"[0, 0]",I bought the triliogy and have tested out all ...,4,Entertaining,1355875200,"12 19, 2012"
3,A2A4COGL9VW2HY,1223000893,Michelle P,"[2, 2]",My female kitty could care less about these vi...,4,Happy to have them,1305158400,"05 12, 2011"
4,A2UBQA85NIGLHA,1223000893,"Tim Isenhour ""Timbo""","[6, 7]","If I had gotten just volume two, I would have ...",3,You really only need vol 2,1330905600,"03 5, 2012"


In [4]:
pd.value_counts(pets_df['overall'])

5    29
4    26
1    20
3    16
2     9
Name: overall, dtype: int64

In [5]:
class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = "NEGATIVE" if score <= 2 else "NEUTRAL" if score <= 3 else "POSITIVE"

In [6]:
import contractions
import string
import re


def clean_text(text):
    text = contractions.fix(text)
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    stopwords = [word.strip() for word in open('stopwords_en.txt')]
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

In [37]:
import json

reviews = []

with open('pet_supplies_10000.json', 'r') as in_file:
    for line in in_file:
        data = json.loads(line)
        text = data['reviewText']
        score = data['overall']
        text = clean_text(text)
        reviews.append(Review(text, score))

In [8]:
reviews[0].text

'purchased trilogy hoping two cats age would interested yr old cat fascinated minutes pictures came got bored year old watched minutes walked away possible wonderful courtyard full greenery trees one neighbors bird feeder enough going outside prefer real life versus taped version likely pass friend cats much wildlife watch mine'

In [9]:
reviews[0].sentiment

'NEUTRAL'

In [10]:
reviews[0].score

3.0

In [29]:
len(reviews)

30000

In [38]:
from sklearn.model_selection import train_test_split

training_data, test_data = train_test_split(reviews, test_size=0.33, random_state=42)

In [31]:
len(training_data)

20100

In [32]:
len(test_data)

9900

In [39]:
train_x = [review.text for review in training_data]
train_y = [review.sentiment for review in training_data]

test_x = [review.text for review in test_data]
test_y = [review.sentiment for review in test_data]

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 3))
train_x_vectors = vectorizer.fit_transform(train_x)

In [41]:
from sklearn.svm import SVC

clf_svm = SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)

SVC(kernel='linear')

In [42]:
test_x_vectors = vectorizer.transform(test_x)

print('The result is:', clf_svm.score(test_x_vectors, test_y))

The result is: 0.8109090909090909


In [43]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression(max_iter=1000)
clf_log.fit(train_x_vectors, train_y)

LogisticRegression(max_iter=1000)

In [44]:
print('The result is:', clf_log.score(test_x_vectors, test_y))

The result is: 0.8181818181818182
